## **# Creacion de Datasets para las APIS**

Vamos a disponibilizar, a la API, los datos limpios que dejaron como resultado nuestros procesos de ETL de los 3 datasets obtenidos de la plataforma **Steam**. 

        - steam_games_limpios.csv
        - user_items_limpios.csv
        - user_reviews_limpios.csv
   
El objetivo es la optimizacion de los recursos de computo y la eficiencia de los procesos.


### **Importamos las librerias** 

In [12]:
import pandas as pd # para el analisis de tablas tabulares

import os # permite interactuar con el sistema operativo

import ast # es útil cuando necesitas analizar o manipular código Python en un nivel más profundo que el proporcionado por el análisis de cadenas.

import numpy as np

In [7]:
steam_games_df = pd.read_parquet('../Dataframes/Dataframes_limpios/steam_games_limpios.parquet')
user_items_df = pd.read_parquet('../Dataframes/Dataframes_limpios/user_items_limpios.parquet')
user_reviews_df = pd.read_parquet('../Dataframes/Dataframes_limpios/user_reviews_limpios.parquet')

In [8]:
steam_games_df.head(2)

,genres,name,tags,specs,price,item_id,developer,year
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,761140,Kotoshiro,2018
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",NaN,643980,Secret Level SRL,2018


In [9]:
user_items_df.head(2)

,item_id,user_id,hours_game
0,10,76561197970982479,0.10
1,30,76561197970982479,0.12


In [10]:
user_reviews_df.head(2)

,item_id,recommend,year,sentiment_analysis
0,10,True,2011,2
1,10,True,2013,1


## **Creacion de DF, especializados en cada Query**
#### nos traemos las columnas y las transformamos, con el objetivo de optimizar las consultas y solo enviar a la API los datos que son necesarios para cada proceso. 

.


### 1. def **PlayTimeGenre( genero : str )**: Debe devolver año con mas horas jugadas para dicho género.

Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}


In [17]:

# Merge de los DataFrames
df_merged = pd.merge(user_items_df, steam_games_df[['item_id', 'year', 'genres']], on='item_id', how='inner')

# Explotar las listas en filas independientes
df_merged = df_merged.explode('genres').reset_index(drop=True)

# Agrupamos por año de lanzamiento y género y sumamos las horas jugadas
df_merged = df_merged.groupby(['year', 'genres'], as_index=False)['hours_game'].sum()

# Ordenar el DataFrame resultante
df_merged = df_merged.sort_values(by=['year', 'genres'])

# Guardar el resultado en un archivo CSV
df_merged.to_parquet('../Dataframes/Api_files/PlayTimeGenre.parquet', index=False)


In [15]:
df_merged

,year,genres,hours_game
0,1983,Action,57.92
1,1983,Adventure,57.92
2,1983,Casual,57.92
3,1984,Action,6.40
4,1984,Adventure,6.40
...,...,...,...
335,2018,Adventure,0.35
336,2018,Early Access,102.12
337,2018,Indie,38.78
338,2018,RPG,63.69


###  2. def **UserForGenre( genero : str )**: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [53]:
df_game = steam_games_df.drop(['name', 'developer', 'price', 'tags','specs'],axis=1)

# Merge de los DataFrames
merged_data=pd.merge(user_items_df,df_game,left_on='item_id', right_on='item_id', how='right')

#borramos item_id, no es relevante
merged_data.drop(['item_id'], axis=1, inplace=True)

#borramos nulos
merged_data.dropna(inplace=True)

#cambiamos el tipo de dato de hours game a int-- nos da las horas. Igual con Year
merged_data['hours_game'] = merged_data['hours_game'].astype(int)
merged_data['year'] = merged_data['year'].astype(int)

#Convertimos la columna genres a tipo lista con el modulo ast.
#merged_data['genres'] = merged_data['genres'].apply(ast.literal_eval)

#Expandimos las listas en columnas
merged_data = merged_data.explode('genres').reset_index(drop=True)

#Encontrar el jugador con más horas jugadas por género y año
max_hours_df = merged_data.loc[merged_data.groupby(['genres', 'year'])['hours_game'].idxmax()]

#Damos un orden a las columnas 
max_hours_df = max_hours_df[['genres', 'year', 'user_id', 'hours_game']]

#Segunda agrupación para obtener las horas jugadas por año para el usuario seleccionado
max_hours = max_hours_df.groupby(['genres', 'year', 'user_id'])['hours_game'].sum().reset_index()

max_hours.to_csv('../Dataframes/Api_files/UserForGenre.parquet', index=False)


In [51]:
merged_data

,user_id,hours_game,genres,year
0,UTNerd24,0,Action,1997
1,UTNerd24,0,Indie,1997
2,UTNerd24,0,Racing,1997
3,saint556,0,Action,1997
4,saint556,0,Indie,1997
...,...,...,...,...
6845688,76561198208507532,3,Action,2004
6845689,76561198221578852,4,Action,2004
6845690,EnVyIsSmexy,0,Action,2004
6845691,76561198283312749,0,Action,2004


In [52]:
max_hours_df

,genres,year,user_id,hours_game
1164917,Action,1983,76561197966936422,5
1206144,Action,1984,76561198064710856,1
4215868,Action,1988,DownSyndromeKid,11
4270980,Action,1989,starfleetbrat,8
6725050,Action,1990,76561198041356854,23
...,...,...,...,...
1398380,Web Publishing,2013,76561198059330972,1090
4497489,Web Publishing,2014,Dark_samaCH_PONY,94
3460781,Web Publishing,2015,Xyphien,1183
3178395,Web Publishing,2016,76561198146189832,2


###  3. **def UsersRecommend( año : int )**: Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [56]:
# Merge de los DataFrames
df_merged = pd.merge(user_reviews_df, steam_games_df, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
result = df_merged.loc[(df_merged['recommend'] == True) & (df_merged['sentiment_analysis'].isin([1, 2])),
                           ['year_x', 'name']]

# Renombrar la columna 'year_x' a 'year'
result = result.rename(columns={'year_x': 'year'})

# Agrupar por 'year' y 'name' y contar las ocurrencias
grouped_result = result.groupby(['year', 'name']).size().reset_index(name='count')

# Ordenar por 'year' y 'count' en orden descendente
grouped_result = grouped_result.sort_values(by=['year', 'count'], ascending=[False, False])

# Obtener el top 3 por año
top3_by_year = grouped_result.groupby('year').head(3)

# Guardar el resultado en un archivo CSV
top3_by_year.to_parquet('../Dataframes/Api_files/UsersRecommend.parquet', index=False)

In [149]:
result

,year,name
0,2011,Counter-Strike
1,2013,Counter-Strike
2,2014,Counter-Strike
3,2014,Counter-Strike
4,2014,Counter-Strike
...,...,...
44400,2015,Spiral Knights
44402,2015,Spiral Knights
44403,2015,Spiral Knights
44405,2011,Puzzle Pirates


In [57]:
top3_by_year

,year,name,count
2719,2015,Counter-Strike: Global Offensive,1166
3636,2015,Team Fortress 2,550
2963,2015,Garry's Mod,310
2177,2014,Team Fortress 2,884
1134,2014,Counter-Strike: Global Offensive,643
1424,2014,Garry's Mod,462
793,2013,Team Fortress 2,453
554,2013,Garry's Mod,224
498,2013,Dota 2,147
329,2012,Team Fortress 2,183


In [150]:
grouped_result

,year,name,count
2711,2015,Counter-Strike: Global Offensive,1164
3628,2015,Team Fortress 2,550
2954,2015,Garry's Mod,312
2808,2015,Dota 2,229
3794,2015,Unturned,229
...,...,...,...
25,2010,Super Meat Boy,1
27,2010,Tom Clancy's Splinter Cell Chaos Theory®,1
28,2010,Trackmania United Forever Star Edition,1
29,2010,Train Simulator,1


In [151]:
top3_by_year

,year,name,count
2711,2015,Counter-Strike: Global Offensive,1164
3628,2015,Team Fortress 2,550
2954,2015,Garry's Mod,312
2171,2014,Team Fortress 2,884
1132,2014,Counter-Strike: Global Offensive,636
1421,2014,Garry's Mod,465
791,2013,Team Fortress 2,459
552,2013,Garry's Mod,224
496,2013,Dota 2,144
329,2012,Team Fortress 2,183


### 4. def **UsersNotRecommend( año : int )**: Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
   
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [58]:
# Merge de los DataFrames
df_merged = pd.merge(user_reviews_df, steam_games_df, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
result = df_merged.loc[(df_merged['recommend'] == True) & (df_merged['sentiment_analysis'].isin([0])),
                           ['year_x', 'name']]

# Renombrar la columna 'year_x' a 'year'
result = result.rename(columns={'year_x': 'year'})

# Agrupar por 'year' y 'name' y contar las ocurrencias
grouped_result = result.groupby(['year', 'name']).size().reset_index(name='count')

# Ordenar por 'year' y 'count' en orden descendente
grouped_result = grouped_result.sort_values(by=['year', 'count'], ascending=[False, False])

# Obtener el top 3 por año menos recomendados
top3_least_recommended_by_year = grouped_result.groupby('year').head(3)

# Guardar el resultado en un archivo CSV
top3_least_recommended_by_year.to_parquet('../Dataframes/Api_files/UsersNotRecommend.parquet', index=False)

In [153]:
result

,year,name
12,2014,Counter-Strike
14,2014,Counter-Strike
42,2014,Call of Duty: World at War
55,2015,Call of Duty: World at War
63,2015,Call of Duty: World at War
...,...,...
44341,2014,Spiral Knights
44362,2014,Spiral Knights
44389,2015,Spiral Knights
44393,2015,Spiral Knights


In [154]:
grouped_result

,year,name,count
943,2015,Counter-Strike: Global Offensive,250
987,2015,Dota 2,52
1369,2015,Team Fortress 2,50
1279,2015,Rust,47
1056,2015,Garry's Mod,45
...,...,...,...
1,2010,Empire: Total War™,1
2,2010,Killing Floor,1
3,2010,Madballs in Babo:Invasion,1
4,2010,Mirror's Edge™,1


In [61]:
top3_least_recommended_by_year

,year,name,count
942,2015,Counter-Strike: Global Offensive,250
986,2015,Dota 2,54
1368,2015,Team Fortress 2,50
374,2014,Counter-Strike: Global Offensive,101
740,2014,Team Fortress 2,95
394,2014,DayZ,84
249,2013,Team Fortress 2,31
158,2013,Garry's Mod,24
140,2013,Dota 2,20
85,2012,Team Fortress 2,21


### 5. def **sentiment_analysis( año : int )**: Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [59]:
Steam_modif= steam_games_df.copy()

Steam_modif.rename(columns={'year': 'year_released'}, inplace=True)

# Merge de los DataFrames
merged_df = pd.merge(Steam_modif, user_reviews_df, on='item_id', how='inner')

# Agrupar por año_released y sentimiento, y contar las ocurrencias
result_df = merged_df.groupby(['year_released', 'sentiment_analysis']).size().reset_index(name='count')

# Pivoteando el DataFrame para tener sentimientos como columnas
result_df = result_df.pivot(index='year_released', columns='sentiment_analysis', values='count').reset_index()

# Renombrar las columnas según el formato deseado
result_df.columns = ['year_released', 'Negative', 'Neutral', 'Positive']

# Rellenar NaN con 0
result_df = result_df.fillna(0)

# Convertir las columnas a tipo int
result_df[['Negative', 'Neutral', 'Positive']] = result_df[['Negative', 'Neutral', 'Positive']].astype(int)

# Guardar el resultado en un archivo CSV
result_df.to_parquet('../Dataframes/Api_files/sentiment_analysis.parquet', index=False)

In [60]:
result_df

,year_released,Negative,Neutral,Positive
0,1989,0,0,1
1,1990,1,0,4
2,1991,0,0,1
3,1992,1,0,2
4,1993,1,0,3
5,1994,3,3,9
6,1995,5,0,10
7,1996,0,1,4
8,1997,8,3,31
9,1998,12,9,54
